# SAM on RGB image (updated Aug 11, 2025)

1 - Import library

In [1]:
import os
import arcpy
from pathlib import Path
from arcpy.sa import *
from arcpy.ia import *


2 - Input/Output folder

In [2]:
from pathlib import Path

project_root = Path.cwd().parent

# Select province
provName = 'SK'  # AB, SK, MB
if provName == 'AB':
    prov_full = 'Alberta'
elif provName == 'SK':
    prov_full = 'Saskatchewan'
elif provName == 'MB':
    prov_full = 'Manitoba'
else:
    raise ValueError("Invalid province name. Use 'AB', 'SK', or 'MB'.")

in_folder = str(project_root / '5_Data' / 'RGB_download' / prov_full)
out_folder = str(project_root / '5_Data' / 'Segmentation' / prov_full)
os.makedirs(out_folder, exist_ok=True)
arcpy.env.workspace = in_folder

3 - Input pre-trained model

In [3]:
sam_model = str(project_root / '5_Data' / 'Models' / 'SAM.dlpk')
batch_options = "padding 256;batch_size 10;box_nms_thresh 0.5;points_per_batch 64;stability_score_thresh 0.5;min_mask_region_area 5000","NMS","Confidence","Class",0,"PROCESS_AS_MOSAICKED_IMAGE"

arcpy.CheckOutExtension("ImageAnalyst")
arcpy.env.parallelProcessingFactor = "80%"
arcpy.env.overwriteOutput = True
rasterList = arcpy.ListRasters("*.tif")

4 - SAM segmentation

In [4]:
# --- config ---
arcpy.env.overwriteOutput = False
os.makedirs(out_folder, exist_ok=True)

for raster in rasterList:
    stem = Path(raster).stem
    out_fc = os.path.join(out_folder, f"Boundary_{stem}.shp")

    # 1) skip if output exists
    if arcpy.Exists(out_fc):
        print(f"Skip (exists): {out_fc}")
        continue
    # 2) process
    print(f"Processing: {raster} -> {out_fc}")
    arcpy.ia.DetectObjectsUsingDeepLearning(raster, out_fc, sam_model, batch_options)

print("All done!")


Processing: rgb_SK_2021_10_1.tif -> c:\Users\spn733\Work\CSA_Field_Boundary_Segmentation_V2\CSA_Field_Boundary_Segmentation\5_Data\Segmentation\Saskatchewan\Boundary_rgb_SK_2021_10_1.shp
Processing: rgb_SK_2021_11_1.tif -> c:\Users\spn733\Work\CSA_Field_Boundary_Segmentation_V2\CSA_Field_Boundary_Segmentation\5_Data\Segmentation\Saskatchewan\Boundary_rgb_SK_2021_11_1.shp
Processing: rgb_SK_2021_11_1.tif -> c:\Users\spn733\Work\CSA_Field_Boundary_Segmentation_V2\CSA_Field_Boundary_Segmentation\5_Data\Segmentation\Saskatchewan\Boundary_rgb_SK_2021_11_1.shp
Processing: rgb_SK_2021_12_1.tif -> c:\Users\spn733\Work\CSA_Field_Boundary_Segmentation_V2\CSA_Field_Boundary_Segmentation\5_Data\Segmentation\Saskatchewan\Boundary_rgb_SK_2021_12_1.shp
Processing: rgb_SK_2021_12_1.tif -> c:\Users\spn733\Work\CSA_Field_Boundary_Segmentation_V2\CSA_Field_Boundary_Segmentation\5_Data\Segmentation\Saskatchewan\Boundary_rgb_SK_2021_12_1.shp
Processing: rgb_SK_2022_10_1.tif -> c:\Users\spn733\Work\CSA_Fiel